In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
import csv
import os
import numpy as np
import sys
from sklearn.model_selection import train_test_split
import importlib
from torchsummary import summary

# Import constants and catch any import errors
from link_to_constants import *
link_constants()
try:
    from constants import * # type: ignore
    print("Successfully imported constants.")
except ImportError as e:
    print(f"Error importing constants: {e}")

Current directory: c:\Users\marco\Documents\GitHub\Tesi\Codice_carla_server\final0911\neural_network
Parent directory: c:\Users\marco\Documents\GitHub\Tesi\Codice_carla_server\final0911
Data gen and processing directory: c:\Users\marco\Documents\GitHub\Tesi\Codice_carla_server\final0911\data_gen_and_processing
Successfully imported constants.


In [2]:
def load_points_grid_map(csv_file):
    """Load bounding box vertices from a CSV file."""
    points = []
    with open(csv_file, 'r') as file:
        reader = csv.reader(file)
        for row in reader:
            # Extract the 3D coordinates of the 8 bounding box vertices
            coordinates = [
                float(row[0]), float(row[1]), float(row[2])
                ]
            points.append(coordinates)
    np_points = np.array(points)
    return np_points

def generate_grid_map (grid_map_path):
    grid_map_files = [f for f in os.listdir(grid_map_path)]
    list_grid_maps = []

    for file in grid_map_files:
        complete_path = os.path.join(grid_map_path, file)
        print(f"Loading {file}...")
        points = load_points_grid_map(complete_path)

        # Recreate the grid map from positions array
        grid_map_recreate = np.full((Y_RANGE, X_RANGE), FLOOR_HEIGHT, dtype=float) # type: ignore

        # Fill the grid map with values from positions array
        for pos in points:
            col, row, height = pos
            grid_map_recreate[int(row), int(col)] = height

        list_grid_maps.append(grid_map_recreate)
    
    return list_grid_maps

def load_points_grid_map_BB (csv_file):
    """Load bounding box vertices from a CSV file."""
    points = []
    with open(csv_file, 'r') as file:
        reader = csv.reader(file)
        next(reader)  # Skip header
        
        for row in reader:
            # Extract the 3D coordinates of the 8 bounding box vertices
            coordinates = [ [ float(row[i]), float(row[i+1]), float(row[i+2]) ] for i in range(2, 12, 3)]
            points.append(coordinates)

    np_points = np.array(points)
    return np_points

def generate_grid_map_BB (grid_map_path):
    grid_map_files = [f for f in os.listdir(grid_map_path)]
    list_grid_maps = []
    list_num_BB = []    

    for file in grid_map_files:
        complete_path = os.path.join(grid_map_path, file)
        print(f"Loading {file}...")
        points = load_points_grid_map_BB(complete_path)

        num_BB = points.shape[0]

        # Recreate the grid map from positions array
        grid_map_recreate = np.full((Y_RANGE, X_RANGE), FLOOR_HEIGHT, dtype=float) # type: ignore

        # Fill the grid map with values from positions array
        for i in range (len(points)):
            for j in range(4):
                col, row, height = points[i][j]
                grid_map_recreate[int(row), int(col)] = height

        list_grid_maps.append(grid_map_recreate)
        list_num_BB.append(num_BB) #used to stratify the dataset
        
    return list_grid_maps, list_num_BB

def split_data(lidar_data, BB_data, num_BB, size):
    # Split the dataset into a combined training and validation set, and a separate test set using num_BB as stratification
    X_train_val, X_test, y_train_val, y_test, num_BB_train_val, num_BB_test = train_test_split(
        lidar_data, # Samples
        BB_data, # Labels
        num_BB, # Number of BB
        test_size = size,
        random_state=SEED, # type: ignore
        stratify=num_BB
    )
    return X_train_val, X_test, y_train_val, y_test, num_BB_train_val, num_BB_test

In [3]:
complete_grid_maps = []
complete_grid_maps_BB = []
complete_num_BB = []

# Load sensor1
grid_maps = generate_grid_map(LIDAR_1_GRID_DIRECTORY) # type: ignore
grid_maps_BB, num_BB  = generate_grid_map_BB(NEW_POSITIONS_LIDAR_1_GRID_DIRECTORY) # type: ignore

complete_grid_maps.append(grid_maps)
complete_grid_maps_BB.append(grid_maps_BB)
complete_num_BB.append(num_BB)

# Load sensor2
grid_maps = generate_grid_map(LIDAR_2_GRID_DIRECTORY) # type: ignore
grid_maps_BB, num_BB = generate_grid_map_BB(NEW_POSITIONS_LIDAR_2_GRID_DIRECTORY) # type: ignore

complete_grid_maps.append(grid_maps)
complete_grid_maps_BB.append(grid_maps_BB)
complete_num_BB.append(num_BB)

# Load sensor3
grid_maps = generate_grid_map(LIDAR_3_GRID_DIRECTORY) # type: ignore
grid_maps_BB, num_BB = generate_grid_map_BB(NEW_POSITIONS_LIDAR_3_GRID_DIRECTORY) # type: ignore

complete_grid_maps.append(grid_maps)
complete_grid_maps_BB.append(grid_maps_BB)
complete_num_BB.append(num_BB)

Loading 20250210_151407_014931_0.csv...
Loading 20250210_151407_041868_1.csv...
Loading 20250210_151407_099691_2.csv...
Loading 20250210_151407_129381_3.csv...
Loading 20250210_151407_177681_4.csv...
Loading 20250210_151407_209894_5.csv...
Loading 20250210_151407_258458_6.csv...
Loading 20250210_151407_302534_7.csv...
Loading 20250210_151407_348506_8.csv...
Loading 20250210_151407_395559_9.csv...
Loading 20250210_151407_425651_10.csv...
Loading 20250210_151407_486779_11.csv...
Loading 20250210_151407_514158_12.csv...
Loading 20250210_151407_580926_13.csv...
Loading 20250210_151407_604882_14.csv...
Loading 20250210_151407_658461_15.csv...
Loading 20250210_151407_690040_16.csv...
Loading 20250210_151407_736634_17.csv...
Loading 20250210_151407_783207_18.csv...
Loading 20250210_151407_815281_19.csv...
Loading 20250210_151407_876351_20.csv...
Loading 20250210_151407_903749_21.csv...
Loading 20250210_151407_970897_22.csv...
Loading 20250210_151407_995557_23.csv...
Loading 20250210_151408_06

In [4]:
del grid_maps, grid_maps_BB, num_BB

In [5]:
print(len(complete_grid_maps[0]), len(complete_grid_maps[1]), len(complete_grid_maps[2]))

1631 426 505


In [6]:
print(len(complete_grid_maps_BB[0]), len(complete_grid_maps_BB[1]), len(complete_grid_maps_BB[2]))

1631 426 505


In [7]:
print(len(complete_num_BB[0]), len(complete_num_BB[1]), len(complete_num_BB[2]))

1631 426 505


In [8]:
# Concatenate the lists in complete_grid_maps along the first dimension
complete_grid_maps = np.concatenate(complete_grid_maps, axis=0)
print(complete_grid_maps.shape)

(2562, 400, 400)


In [9]:
# Concatenate the lists in complete_grid_maps_BB along the first dimension
complete_grid_maps_BB = np.concatenate(complete_grid_maps_BB, axis=0)
print(complete_grid_maps_BB.shape)

(2562, 400, 400)


In [10]:
complete_num_BB = np.concatenate(complete_num_BB, axis=0)
print(complete_num_BB.shape)

(2562,)


In [11]:
complete_num_BB = np.expand_dims(complete_num_BB, axis=1)
print(complete_num_BB.shape)

(2562, 1)


In [12]:
# Split the data
X_train_val, X_test, y_train_val, y_test, num_BB_train_val, num_BB_test = split_data(complete_grid_maps, complete_grid_maps_BB, complete_num_BB, TEST_SIZE) # type: ignore
X_train, X_val, y_train, y_val, num_BB_train, num_BB_val = split_data(X_train_val, y_train_val, num_BB_train_val, len(X_test)) # Esure that val and test set have the same lenght

In [13]:
del complete_grid_maps, complete_grid_maps_BB, complete_num_BB

In [14]:
print(X_train.shape, X_val.shape, X_test.shape, y_train.shape, y_val.shape, y_test.shape, num_BB_train.shape, num_BB_val.shape, num_BB_test.shape)

(2048, 400, 400) (257, 400, 400) (257, 400, 400) (2048, 400, 400) (257, 400, 400) (257, 400, 400) (2048, 1) (257, 1) (257, 1)


In [15]:
sum_train = 0
sum_val = 0
sum_test = 0
for i in range(len(num_BB_train)):
    sum_train += num_BB_train[i]
print(f"Sum_train: ", sum_train)
print(f"Average_sum_train: ",sum_train/len(num_BB_train))

for i in range(len(num_BB_val)):
    sum_val += num_BB_val[i]
print(f"Sum_val: ",sum_val)
print(f"Average_sum_val: ",sum_val/len(num_BB_val))

for i in range(len(num_BB_test)):
    sum_test += num_BB_test[i]
print(f"Sum_test: ",sum_test)
print(f"Average_sum_test: ",sum_test/len(num_BB_test))

Sum_train:  [3539]
Average_sum_train:  [1.72802734]
Sum_val:  [446]
Average_sum_val:  [1.73540856]
Sum_test:  [446]
Average_sum_test:  [1.73540856]


In [16]:
from sklearn.preprocessing import MinMaxScaler

# Initialize the scaler
scaler = MinMaxScaler()

# Normalize the data
X_train = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
X_val = scaler.transform(X_val.reshape(-1, X_val.shape[-1])).reshape(X_val.shape)
X_test = scaler.transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape)

y_train = scaler.fit_transform(y_train.reshape(-1, y_train.shape[-1])).reshape(y_train.shape)
y_val = scaler.transform(y_val.reshape(-1, y_val.shape[-1])).reshape(y_val.shape)
y_test = scaler.transform(y_test.reshape(-1, y_test.shape[-1])).reshape(y_test.shape)

In [17]:
print(X_train.shape, X_val.shape, X_test.shape, y_train.shape, y_val.shape, y_test.shape)

(2048, 400, 400) (257, 400, 400) (257, 400, 400) (2048, 400, 400) (257, 400, 400) (257, 400, 400)


In [18]:
X_train = np.expand_dims(X_train, axis=1)
X_val = np.expand_dims(X_val, axis=1)
X_test = np.expand_dims(X_test, axis=1)
print("New input shape (train, val, test):", X_train.shape, X_val.shape, X_test.shape)

New input shape (train, val, test): (2048, 1, 400, 400) (257, 1, 400, 400) (257, 1, 400, 400)


In [19]:
y_train = np.expand_dims(y_train, axis=1)
y_val = np.expand_dims(y_val, axis=1)
y_test = np.expand_dims(y_test, axis=1)
print("New labels shape (train, val, test):", y_train.shape, y_val.shape, y_test.shape)

New labels shape (train, val, test): (2048, 1, 400, 400) (257, 1, 400, 400) (257, 1, 400, 400)


In [20]:
# Define the autoencoder model
class Autoencoder(nn.Module):
    def __init__(self): # Constructor method for the autoencoder
        super(Autoencoder, self).__init__() # Calls the constructor of the parent class (nn.Module) to set up necessary functionality.
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, stride = 2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, stride = 2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, stride = 2)
        )
        self.decoder = nn.Sequential(
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(64, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(32, 1, kernel_size=3, padding=1),
        )

    def forward(self, x): # The forward method defines the computation that happens when the model is called with input x.
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [21]:
# Prepare data loaders
train_dataset = TensorDataset(torch.from_numpy(X_train).float(), torch.from_numpy(y_train).float()) # Each element in train_dataset will be a tuple (input, target). Both will have shape (400,400). There will be as many elements in the dataset as there are samples in X_train.
val_dataset = TensorDataset(torch.from_numpy(X_val).float(), torch.from_numpy(y_val).float())
test_dataset = TensorDataset(torch.from_numpy(X_test).float(), torch.from_numpy(y_test).float())

In [22]:
del X_train, X_val, X_test, y_train, y_val, y_test

In [23]:
print(len(train_dataset), len(val_dataset), len(test_dataset))

2048 257 257


In [24]:
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [25]:
del train_dataset, val_dataset, test_dataset

In [26]:
class EarlyStopping:
    def __init__(self, patience=5, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.best_loss = None
        self.counter = 0
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif val_loss < self.best_loss - self.min_delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

In [27]:
def weights_init(m):
    if isinstance(m, nn.Conv2d):
        init.kaiming_normal_(m.weight, nonlinearity='relu')
        if m.bias is not None:
            init.constant_(m.bias, 0)
    elif isinstance(m, nn.Linear):
        init.kaiming_normal_(m.weight, nonlinearity='relu')
        if m.bias is not None:
            init.constant_(m.bias, 0)

In [ ]:
model = Autoencoder()
model.apply(weights_init)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.1) # model.parameters() passes the parameters of the model to the optimizer so that it can update them during training.
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=5)

In [44]:
print(f"Is CUDA supported by this system? {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")

# Storing ID of current CUDA device
cuda_id = torch.cuda.current_device()
print(f"ID of current CUDA device:{torch.cuda.current_device()}")
       
print(f"Name of current CUDA device:{torch.cuda.get_device_name(cuda_id)}")

Is CUDA supported by this system? True
CUDA version: 12.4
ID of current CUDA device:0
Name of current CUDA device:NVIDIA GeForce RTX 4050 Laptop GPU


In [45]:
# Check if CUDA is available
if torch.cuda.is_available():
    device = torch.device("cuda")
    model = model.to(device)
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")
else:
    device = torch.device("cpu")
    print("CUDA is not available. Using CPU.")

Using GPU: NVIDIA GeForce RTX 4050 Laptop GPU


In [ ]:
summary(model, (1, 400, 400))

In [ ]:
# Clear GPU cache
torch.cuda.empty_cache()

# Initialize early stopping
early_stopping = EarlyStopping(patience=3, min_delta=0.001)

num_epochs = 20
# Training loop
for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    for data in train_loader:
        inputs, targets = data
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()  # Reset gradients
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    train_loss /= len(train_loader)  # Average loss over all batches

    model.eval()
    val_loss = 0
    with torch.no_grad():
        for data in val_loader:
            inputs, targets = data
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            val_loss += loss.item()
    val_loss /= len(val_loader)  # Average loss over all batches

    # Step the scheduler with the validation loss
    scheduler.step(val_loss)

    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')
    # Clear GPU cache
    torch.cuda.empty_cache()

    # Check early stopping
    early_stopping(val_loss)
    if early_stopping.early_stop:
        print("Early stopping triggered")
        break

# Evaluate on test set
model.eval()
test_loss = 0
with torch.no_grad():
    for data in test_loader:
        inputs, targets = data
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        test_loss += loss.item()
test_loss /= len(test_loader)
print(f'Test Loss: {test_loss:.4f}')
# Clear GPU cache
torch.cuda.empty_cache()


# Make predictions
predictions = []
with torch.no_grad():
    for data in test_loader:
        inputs, _ = data
        inputs= inputs.to(device)
        outputs = model(inputs)
        predictions.append(outputs)
predictions = torch.cat(predictions).cpu().numpy()
print("Predictions Shape:", predictions.shape)
# Clear GPU cache
torch.cuda.empty_cache()


Epoch 1/20, Train Loss: 3036943981277.3638, Val Loss: 0.2270
Epoch 2/20, Train Loss: 0.2253, Val Loss: 0.2241
Epoch 3/20, Train Loss: 0.2230, Val Loss: 0.2221
Epoch 4/20, Train Loss: 0.2213, Val Loss: 0.2206
Epoch 5/20, Train Loss: 0.2200, Val Loss: 0.2195
Epoch 6/20, Train Loss: 0.2190, Val Loss: 0.2182
Epoch 7/20, Train Loss: 0.2174, Val Loss: 0.2167
Epoch 8/20, Train Loss: 0.2149, Val Loss: 0.2137
Epoch 9/20, Train Loss: 0.2130, Val Loss: 0.2124
Epoch 10/20, Train Loss: 0.2112, Val Loss: 0.2100
Epoch 11/20, Train Loss: 0.2095, Val Loss: 0.2091
Epoch 12/20, Train Loss: 0.2089, Val Loss: 0.2086
Epoch 13/20, Train Loss: 0.2084, Val Loss: 0.2083
Epoch 14/20, Train Loss: 0.2081, Val Loss: 0.2079
Epoch 15/20, Train Loss: 0.2078, Val Loss: 0.2077
Epoch 16/20, Train Loss: 0.2075, Val Loss: 0.2074
Epoch 17/20, Train Loss: 0.2073, Val Loss: 0.2072
Epoch 18/20, Train Loss: 0.2071, Val Loss: 0.2070
Epoch 19/20, Train Loss: 0.2069, Val Loss: 0.2068
Epoch 20/20, Train Loss: 0.2066, Val Loss: 0.20